In [1]:
import glob
import json
import random
import pandas
from pprint import pprint
import sys
import json
import numpy as np
import pymssql
from gemd_database import MSSQLDatabase
from queries import *
import SciServer.CasJobs as cj

In [2]:
with open('/home/arachid1/.private/gemd_db.json','r') as f:
    AUTH = json.load(f)

DATABASE='GEMD'
GEMDDB=MSSQLDatabase(AUTH,DATABASE)

In [3]:
sql="""
select distinct *
from GEMDModel
"""
GEMDDB.execute_query(sql)

,id,name
0,1,ali_nicholas_EuSNbSe2_gemd_model_dumps
1,2,soren_NYO_gemd_model_dumps
2,3,laser_shock_gemd_model_dumps


In [4]:
sql="""
select top 4 context
  from  gemdobject c where gemd_type='process_run'
 order by newid()
"""
query_results=GEMDDB.execute_query(sql)
query_results
# query_results.iloc[0]['context']

,context
0,"{""conditions"": [], ""file_links"": [], ""name"": ""..."
1,"{""conditions"": [], ""file_links"": [], ""name"": ""..."
2,"{""conditions"": [], ""file_links"": [], ""name"": ""..."
3,"{""conditions"": [], ""file_links"": [], ""name"": ""..."


find process runs for a random set of process specs

In [5]:
sql="""
with some_specs as (  
select top 10 *              -- using SELECT TOP N ... ORDER BY newid() is standard trick for getting a random sample of N entries
  from gemdobject
 where gemd_type='process_spec' 
order by newid()                       
)
select s.uid as spec_id
,      json_value(s.context,'$.name') as spec
,      r.uid as run_id
,      r.context as run
  from some_specs s
  join gemdobject r
 on r.gemd_type='process_run' 
   and JSON_VALUE(r.context,'$.spec.id')=s.uid
order by spec
"""
query_results=GEMDDB.execute_query(sql)
query_results

,spec_id,spec,run_id,run
0,a16eff2b-08d6-4f90-9621-b67b9315aef8,240um Kapton Original v2 (2021-10-22),2468e0d5-1672-4b75-b8d3-e51a91e891c4,"{""conditions"": [], ""file_links"": [], ""name"": ""..."
1,a16eff2b-08d6-4f90-9621-b67b9315aef8,240um Kapton Original v2 (2021-10-22),249417d3-3f26-4056-99b2-25bcf5fe4c1a,"{""conditions"": [], ""file_links"": [], ""name"": ""..."
2,a16eff2b-08d6-4f90-9621-b67b9315aef8,240um Kapton Original v2 (2021-10-22),24a4bb35-38ba-434d-a395-7addc06dd752,"{""conditions"": [], ""file_links"": [], ""name"": ""..."
3,a16eff2b-08d6-4f90-9621-b67b9315aef8,240um Kapton Original v2 (2021-10-22),24bbcbf6-559c-4ff7-8856-75c4f868ad74,"{""conditions"": [], ""file_links"": [], ""name"": ""..."
4,a16eff2b-08d6-4f90-9621-b67b9315aef8,240um Kapton Original v2 (2021-10-22),250fff1c-99b1-43c1-823b-e7fd65320244,"{""conditions"": [], ""file_links"": [], ""name"": ""..."
...,...,...,...,...
686,c0465c83-ead5-4100-b4b7-2210364cda2e,Impact Sample Cutting and Polishing,772a7834-c2ea-47cd-a90d-58730a0c10a9,"{""conditions"": [], ""file_links"": [], ""name"": ""..."
687,5275f77f-9d42-4782-a3ce-7adc96957aa5,Impact Sample Cutting and Polishing,18d7087e-46ec-4ce1-8f66-ad83e1721472,"{""conditions"": [], ""file_links"": [], ""name"": ""..."
688,f01a27ac-4784-4690-b6c4-a74566ee0006,Impact Sample Cutting and Polishing,5cbca666-3951-47d3-b927-62b7d8e7bfcd,"{""conditions"": [], ""file_links"": [], ""name"": ""..."
689,64a56ff6-9f99-40d0-8e70-5fa81d3cf15d,Pressing Ground EuS and NbSe2 mix,471d9e4f-b402-4755-9bfa-de6a8c0b8978,"{""conditions"": [{""file_links"": [], ""name"": ""At..."


for a material spec (non empty properties), get material runs and templates

In [6]:
%%time
sql="""
with some_specs as (
select top 3 *
  from gemdobject
 where gemd_type='material_spec' 
   and context like '%"properties":%'
and context not like '%properties": [[]]%'   -- having some non-trivial properties. NOTE '[' must be escaped to '[[]', ']' needs no escaping
                                             -- see https://docs.microsoft.com/en-us/sql/t-sql/language-elements/like-transact-sql?view=sql-server-ver16#using-wildcard-characters-as-literals
)
select s.uid as spec_id
,      JSON_VALUE(t.context,'$.name') as template
,      JSON_VALUE(s.context,'$.name') as spec
,      JSON_VALUE(m.context,'$.name') as material
,      JSON_VALUE(p.context,'$.name') as process
,      JSON_VALUE(ps.context,'$.name') as process_spec
--,      t.context as t_context
--,      s.context as s_context
--,      m.context as m_context
,      json_query(ps.context,'$.tags') as ps_tags
--,      p.context as p_context
--,      ps.context as ps_context
  from some_specs s
  join gemdobject t on t.uid=json_value(s.context,'$.template.id') -- get template for material_spec
  join gemdobject m on s.uid=json_value(m.context,'$.spec.id')     -- get material_run for material_spec
  join gemdobject p on p.uid=json_value(m.context,'$.process.id')  -- get process_run for material_run
  join gemdobject ps on ps.uid=json_value(p.context,'$.spec.id')   -- get process_spec for process_run
  """
query_results=GEMDDB.execute_query(sql)
query_results.head(4)

CPU times: user 982 µs, sys: 3.46 ms, total: 4.44 ms
Wall time: 3.1 s


,spec_id,template,spec,material,process,process_spec,ps_tags
0,0012f07a-6ae0-484a-978d-a95832aa5420,Heated Material,Synthesized NbSe2,Synthesized NbSe2,Heating Ground NbSe2 Mixture,Heating Ground NbSe2 Mixture,[]
1,0012f07a-6ae0-484a-978d-a95832aa5420,Heated Material,Synthesized NbSe2,Synthesized NbSe2,Heating Ground NbSe2 Mixture,Heating Ground NbSe2 Mixture,[]
2,0d023f58-1930-409c-9526-55f1cd52c954,Chunked Material,Chunked Europium,Chunked Europium,Chunking Europium,Chunking Europium,[]
3,0d023f58-1930-409c-9526-55f1cd52c954,Chunked Material,Chunked Europium,Chunked Europium,Chunking Europium,Chunking Europium,[]


get properties for a material spec

In [7]:
%%time
sql="""
select s.uid as spec_id
,      JSON_VALUE(s.context,'$.name') as spec
,      JSON_VALUE(sp.property,'$.name') as p
,      sp.property
  from gemdobject s
  cross apply OPENJSON(s.context,'$.properties') with (property nvarchar(max) 'lax$.property' as json) sp
 where s.gemd_type='material_spec' 
  """
query_results=GEMDDB.execute_query(sql)
query_results

CPU times: user 5.97 ms, sys: 481 µs, total: 6.45 ms
Wall time: 43.6 ms


,spec_id,spec,p,property
0,0012f07a-6ae0-484a-978d-a95832aa5420,Synthesized NbSe2,Form,"{""file_links"": [], ""name"": ""Form"", ""notes"": nu..."
1,00ee1881-8d9c-4fa3-8bd1-b86d4aa3c378,Ground EuS and NbSe2 mix,Form,"{""file_links"": [], ""name"": ""Form"", ""notes"": nu..."
2,0d023f58-1930-409c-9526-55f1cd52c954,Chunked Europium,Form,"{""file_links"": [], ""name"": ""Form"", ""notes"": nu..."
3,133a4fac-c0d7-4f20-87ec-95c79bda55bd,baked_Na2CO3,Chemical composition,"{""file_links"": [], ""name"": ""Chemical compositi..."
4,133a4fac-c0d7-4f20-87ec-95c79bda55bd,baked_Na2CO3,Material form,"{""file_links"": [], ""name"": ""Material form"", ""n..."
...,...,...,...,...
126,f0bff219-f11d-423e-a4ae-831375403e1b,"101.6um (0.004"") Hard Al (AluFoil)",FoilHardness,"{""file_links"": [], ""name"": ""FoilHardness"", ""no..."
127,f0bff219-f11d-423e-a4ae-831375403e1b,"101.6um (0.004"") Hard Al (AluFoil)",FoilMaterial,"{""file_links"": [], ""name"": ""FoilMaterial"", ""no..."
128,f9e37bdd-d08f-4538-99f7-9c0eb8ea0da9,"Borosilicate, T=0.747""",GlassThickness,"{""file_links"": [], ""name"": ""GlassThickness"", ""..."
129,f9e37bdd-d08f-4538-99f7-9c0eb8ea0da9,"Borosilicate, T=0.747""",GlassLength,"{""file_links"": [], ""name"": ""GlassLength"", ""not..."


return reachable nodes from uid node

081bbfd7-1bbe-40d4-bb44-15d78b3e23bb is an uid of Impact Sample Cutting and Polishing from the laser shock model

In [8]:
query_results=GEMDDB.execute_query(reachable_nodes_query(uid='081bbfd7-1bbe-40d4-bb44-15d78b3e23bb'))
query_results

,Source,Target
0,process_run [081bbfd7-1bbe-40d4-bb44-15d78b3e2...,process_spec [fd7565da-a410-40ff-b781-1f57a191...
1,process_spec [fd7565da-a410-40ff-b781-1f57a191...,process_template [ee2196b4-b0eb-4c47-b2ff-4ba3...


In [9]:
query_results=GEMDDB.execute_query(to_node_query(model_id=3))
query_results

,endpoint_uid,num_in_nodes
0,None,50071
1,ee2196b4-b0eb-4c47-b2ff-4ba3fa58f25e,11597
2,abc629d5-7453-4620-accd-db59f8586067,9916
3,705961c4-081b-4cfb-ae02-d0c7eb3e2008,9683
4,b6fbde0c-bbf3-44fc-ada9-ebd2dd07f69b,8409
...,...,...
26692,b58dec27-736e-405b-9961-cca02d5af9a3,1
26693,0f1c1ce0-e4af-456f-a379-8c50ccfac352,1
26694,f290499f-140e-4e8c-88ba-809de92bbaf0,1
26695,67214940-7f7f-4da4-b785-7611e64ea659,1


In [10]:
# sql="""
# select distinct *
# from GEMDModel
# """
# ee2196b4-b0eb-4c47-b2ff-4ba3fa58f25e
# GEMDDB.execute_query(sql)

In [11]:
query_results=GEMDDB.execute_query(multiple_paths_nodes_query(model_id=3))
query_results

,root_uid,root_type,endpoint_uid,endpoint_type,path,min_level,max_level,num_paths
0,000043d9-701d-4a81-bf62-4c1cb1cca72f,material_run,b6fbde0c-bbf3-44fc-ada9-ebd2dd07f69b,process_template,material_run000043d9-701d-4a81-bf62-4c1cb1cca7...,3,3,2
1,000043d9-701d-4a81-bf62-4c1cb1cca72f,material_run,b9274d86-b97a-4e39-b097-efee075e171a,process_spec,material_run000043d9-701d-4a81-bf62-4c1cb1cca7...,2,2,2
2,00027938-31ce-4580-af14-d8626e735d35,material_run,b6fbde0c-bbf3-44fc-ada9-ebd2dd07f69b,process_template,material_run00027938-31ce-4580-af14-d8626e735d...,3,3,2
3,00027938-31ce-4580-af14-d8626e735d35,material_run,f1556630-0f5c-4996-b016-36a4c822f9cb,process_spec,material_run00027938-31ce-4580-af14-d8626e735d...,2,2,2
4,0007c08e-c7f1-4568-bedc-543ebbc060df,material_run,8a150e80-4108-4ecf-9184-00000b40d06e,process_spec,material_run0007c08e-c7f1-4568-bedc-543ebbc060...,2,2,2
...,...,...,...,...,...,...,...,...
24055,ffdf443e-eed2-4c9a-a5ea-7ce1dd11ffc3,material_run,ee2196b4-b0eb-4c47-b2ff-4ba3fa58f25e,process_template,material_runffdf443e-eed2-4c9a-a5ea-7ce1dd11ff...,3,3,2
24056,ffe704e6-6c4f-4bfb-8930-7e87a61a0261,material_run,d5557c35-5104-4480-8a46-f093f8e8a4bb,process_template,material_runffe704e6-6c4f-4bfb-8930-7e87a61a02...,3,3,2
24057,ffe704e6-6c4f-4bfb-8930-7e87a61a0261,material_run,d6d849c4-5a00-4176-8b7c-0e0ac276e32e,process_spec,material_runffe704e6-6c4f-4bfb-8930-7e87a61a02...,2,2,2
24058,fff2d0fe-a165-4314-bb57-6608bc17c4f7,material_run,ee2196b4-b0eb-4c47-b2ff-4ba3fa58f25e,process_template,material_runfff2d0fe-a165-4314-bb57-6608bc17c4...,3,3,2


In [12]:
query_results=GEMDDB.execute_query(multiple_paths_nodes_query(model_id=3))
query_results

,root_uid,root_type,endpoint_uid,endpoint_type,path,min_level,max_level,num_paths
0,000043d9-701d-4a81-bf62-4c1cb1cca72f,material_run,b6fbde0c-bbf3-44fc-ada9-ebd2dd07f69b,process_template,material_run000043d9-701d-4a81-bf62-4c1cb1cca7...,3,3,2
1,000043d9-701d-4a81-bf62-4c1cb1cca72f,material_run,b9274d86-b97a-4e39-b097-efee075e171a,process_spec,material_run000043d9-701d-4a81-bf62-4c1cb1cca7...,2,2,2
2,00027938-31ce-4580-af14-d8626e735d35,material_run,b6fbde0c-bbf3-44fc-ada9-ebd2dd07f69b,process_template,material_run00027938-31ce-4580-af14-d8626e735d...,3,3,2
3,00027938-31ce-4580-af14-d8626e735d35,material_run,f1556630-0f5c-4996-b016-36a4c822f9cb,process_spec,material_run00027938-31ce-4580-af14-d8626e735d...,2,2,2
4,0007c08e-c7f1-4568-bedc-543ebbc060df,material_run,8a150e80-4108-4ecf-9184-00000b40d06e,process_spec,material_run0007c08e-c7f1-4568-bedc-543ebbc060...,2,2,2
...,...,...,...,...,...,...,...,...
24055,ffdf443e-eed2-4c9a-a5ea-7ce1dd11ffc3,material_run,ee2196b4-b0eb-4c47-b2ff-4ba3fa58f25e,process_template,material_runffdf443e-eed2-4c9a-a5ea-7ce1dd11ff...,3,3,2
24056,ffe704e6-6c4f-4bfb-8930-7e87a61a0261,material_run,d5557c35-5104-4480-8a46-f093f8e8a4bb,process_template,material_runffe704e6-6c4f-4bfb-8930-7e87a61a02...,3,3,2
24057,ffe704e6-6c4f-4bfb-8930-7e87a61a0261,material_run,d6d849c4-5a00-4176-8b7c-0e0ac276e32e,process_spec,material_runffe704e6-6c4f-4bfb-8930-7e87a61a02...,2,2,2
24058,fff2d0fe-a165-4314-bb57-6608bc17c4f7,material_run,ee2196b4-b0eb-4c47-b2ff-4ba3fa58f25e,process_template,material_runfff2d0fe-a165-4314-bb57-6608bc17c4...,3,3,2
